In [ ]:
import os
import sys
import re
import json
import pandas as pd 
import numpy as np
import seaborn as sns; sns.set_theme(style="whitegrid");
import matplotlib.pyplot as plt

# We are including our repository bin to the system path so that we can import the following python modules
bin_path = os.path.abspath(os.path.join('../../bin'))
if bin_path not in sys.path:
    sys.path.append(bin_path)
    
import run_gnssrefl 
import gnssrefl_helpers

%matplotlib inline

In [ ]:
#Making sure environment variables are set - this is required to run the gnssrefl code
exists = gnssrefl_helpers.check_environment()
if exists == False:
    gnssrefl_helpers.set_environment()
else:
     print('environment variable ORBITS path is', os.environ['ORBITS'],
          '\nenvironment variable REFL_CODE path is', os.environ['REFL_CODE'],
          '\nenvironment variable EXE path is', os.environ['EXE'])
        
refl_code_loc = os.environ['REFL_CODE']
# import the crx2rnx file which is dependant on your working OS - this is required to run the gnssrefl code
gnssrefl_helpers.download_crx2rnx()

# Dye2, Greenland 

**Station Name:** gls1

**Location:**  Dye2, Qeqqata Province, Greenland 

**Archive:**  [UNAVCO](http://www.unavco.org), [SOPAC](http://sopac-csrc.ucsd.edu/index.php/sopac/)

**DOI:**  [https://doi.org/10.7283/T5WS8RDB](https://doi.org/10.7283/T5WS8RDB)

**Ellipsoidal Coordinates:**

- Latitude: 66.47940

- Longitude:  -46.31015

- Height: 2150 m

[Station Page at UNAVCO](https://www.unavco.org/instrumentation/networks/status/nota/overview/gls1)

[Station Page at Nevada Geodetic Laboratory](http://geodesy.unr.edu/NGLStationPages/stations/GLS1.sta)

[Google Maps Link](https://goo.gl/maps/391a7h2HpacAa59u8) 

<img src="../../data/gls1-photo.png" width="400">
<img src="../../data/gls1.jpg" width="400">
<BR>


## Data Summary

Station gls1 was installed at [Dye2](http://greenlandtoday.com/dye-2-a-relic-from-a-not-so-distant-past/?lang=en) on the Greenland Ice Sheet in 2011. 
The antenna is mounted on a long pole; approximately 3.5-meter of the pole was above the ice at the time of installation. 
The receiver at the site only consistently tracks legacy GPS signals. A detailed discussion of the monument and 
data from the station can be found in [Larson, MacFerrin, and Nylen (2020)](https://tc.copernicus.org/articles/14/1985/2020/tc-14-1985-2020.pdf). 
The latest position time series for gls1 can be retrieved 
from the [Nevada Geodetic Laboratory](http://geodesy.unr.edu/gps_timeseries/tenv3/IGS14/GLS1.tenv3). 
We also have a utility you can use: **download_unr**

As gls1 is on an ice sheet and the ice surface is relatively smooth in all directions, it 
is unlikely that a complicated azimuth mask will be required.
gls1 was originally installed with an elevation mask of 7 degrees, which is suboptimal for reflections research.
Even though the mask was later removed, we will use 7 degrees as the minimum elevation angle for all our analysis.
Similarly, even though the site managers later changed to enable L2C tracking, to ensure that 
a consistent dataset is being used, we will only use L1 data. 

In [ ]:
%%html
<iframe src="https://gnss-reflections.org/api?example=gls1" width="800" height="950"></iframe>

## quickLook 

Our ultimate goal in this use case is to analyze one year of data. We have chosen the year 
2012 because there was a large melt event on the ice sheet. In order to set the proper
quality control parameters, we will use **quickLook** for one day. First we need to translate 
one day of RINEX data using **rinex2snr**. We will use day of year 100:

In [ ]:
station = 'gls1'
year = 2012 
doy = 100

lat = 66.4794
long = -46.3102
height = 2148.578

In [ ]:
# To understand what rinex2snr returns, you can uncomment the next line of code to learn more about this function 
# and it's default parameters
# check_parameters.rinex2snr?
run_gnssrefl.rinex2snr(station, year, doy, translator='hybrid')

In [ ]:
values, metrics = run_gnssrefl.quicklook(station, year, doy=doy)

fig, axes = plt.subplots(ncols=2, nrows=2, figsize=(10,10), sharex=True)
fig.suptitle(f'QuickLook: {station}', size=16)

quadrants = ['NW', 'NE', 'SW', 'SE']

for i, ax in enumerate(axes.flat):
    for fail_satellite in values[f'f{quadrants[i]}']:
        g = sns.lineplot(x=values[f'f{quadrants[i]}'][fail_satellite][0], y=values[f'f{quadrants[i]}'][fail_satellite][1], ax=ax, color='lightgrey')
    for satellite in values[quadrants[i]]:
        g = sns.lineplot(x=values[quadrants[i]][satellite][0], y=values[quadrants[i]][satellite][1], ax=ax)
    g.set_ylabel('volts/volts')
    g.set_xlabel('reflector height (m)')

plt.tight_layout()
plt.show()

This is a geographically oriented-summary of the frequency content of the GPS data.
The peaks in these periodograms tell us how high the GPS antenna is above the ice surface.
The peaks are associated with a reflector height (RH) of ~2.5 meters. [For more details on quicklook output](https://github.com/kristinemlarson/gnssrefl/blob/master/docs/quickLook_desc.md).

The next plot shows results with respect to azimuth angle.  The top plot is RH and the other 
two are quality control measures: peak amplitude and peak to noise ratio.


In [ ]:
success, fail = gnssrefl_helpers.quicklook_metrics(metrics)
fig, axes = plt.subplots(ncols=1, nrows=3, figsize=(10,10), sharex=True)
fig.suptitle(f'QuickLook Retrieval Metrics: {station} GPS L1', size=16)

for i, ax in enumerate(axes):
    g = sns.scatterplot(x='Azimuth',y=success.columns[i+1], data=success, ax=ax, label='good')
    g = sns.scatterplot(x='Azimuth',y=fail.columns[i+1], data=fail, ax=ax, color='lightgrey', label='bad')
    
axes[0].legend(loc='upper right')

avg_rh = np.mean(success['Reflector Height'])
qc_val_peak2noise = round(min(success['Peak to Noise']))
axes[1].axhline(qc_val_peak2noise, linestyle='--', color='black', label='QC value used')
qc_val_amp = round(min(success['Amplitude']))
axes[2].axhline(qc_val_amp, linestyle='--', color='black', label='QC value used')
print(f'Average reflector height value: {avg_rh:.1f}')
print('QC value for peak to noise:', qc_val_peak2noise)
print('QC value for amplitude:', qc_val_amp)

plt.tight_layout()
plt.show()

In the top plot we see that the retrieved reflector heights are consistent at all azimuths.
Retrievals for azimuths between 340 degrees and 40 degrees are consistently marked as not having
met quality control settings.From the center plot we can see that a peak2noise QC metric of 3 is reasonable. 
Similarly, the amplitudes (bottom plot) are generally larger than 10, so 8 is an acceptable minimum value.

Compare that to its level when the site was installed in the year 2011:

In [ ]:
run_gnssrefl.rinex2snr(station, year=2011, doy=271, archive='unavco', translator='hybrid')


values, metrics = run_gnssrefl.quicklook(station, 2011, doy=271)
fig, axes = plt.subplots(ncols=2, nrows=2, figsize=(10,10), sharex=True)
fig.suptitle(f'QuickLook: {station}', size=16)

for i, ax in enumerate(axes.flat):
    for fail_satellite in values[f'f{quadrants[i]}']:
        g = sns.lineplot(x=values[f'f{quadrants[i]}'][fail_satellite][0], y=values[f'f{quadrants[i]}'][fail_satellite][1], ax=ax, color='lightgrey')
    for satellite in values[quadrants[i]]:
        g = sns.lineplot(x=values[quadrants[i]][satellite][0], y=values[quadrants[i]][satellite][1], ax=ax)
    g.set_ylabel('volts/volts')
    g.set_xlabel('reflector height (m)')

plt.tight_layout()
plt.show()

The first step is to make SNR files for the year 2012:

This should take ~5 minutes

In [ ]:
run_gnssrefl.rinex2snr(station,year,1, doy_end=366, translator='hybrid')

We will next analyze a year of L1 GPS reflection data from this site. We will use the default minimum and maximum 
reflector height values (0.4 and 6 meters). But for the reasons previously stated, we will set a minimum elevation angle 
of 7 degrees. We also specify that we only want to use the L1 data and set peak2noise and a mimimum
amplitude for the periodograms. We use the utility **make_json_input** to set and store these analysis settings:

In [ ]:
run_gnssrefl.make_json(station, lat, long, height, e1=7, peak2noise=3, ampl=8, l1=True)

In [ ]:
# This is the json file that was created
json_file = f'{refl_code_loc}/input/gls1.json'
with open(json_file, "r") as myfile:
    file = json.load(myfile)
    file['azval'] = [40,90,90,180,180,270,270,330]
os.remove(json_file)
with open(json_file, 'w') as f:
    json.dump(file, f, indent=4)
    
with open(json_file, "r") as myfile:
    file = json.load(myfile)

file

We have also excluded a bit of the northern tracks by hand-editing the json. This is not required as 
the software appears to be appropriately removing these unreliable azimuths. Note: the removal of these
azimuths is more related to the GPS satellite inclination than local conditions at gls1.

Now that you have SNR files and json inputs, you can go ahead and estimate reflector heights for the year 2012:

In [ ]:
#run_gnssrefl.gnssir?

In [ ]:
year = 2012
doy = 1
doy_end = 366
plot=False
run_gnssrefl.gnssir(station, year, doy, doy_end=doy_end, plt=plot, screenstats=False)

We will use the **daily_avg** tool to compute a daily average RH. A median filter is set to 0.25 meters 
and 30 individual tracks are required in order to recover a daily average:

In [ ]:
run_gnssrefl.daily_avg(station, medfilter=.25, ReqTracks=30, plt2screen=False, txtfile='gls1-dailyavg.txt')

In [ ]:
filepath = f'{refl_code_loc}/Files/{station}_allRH.txt'
data = gnssrefl_helpers.read_allrh_file(filepath)

df = pd.DataFrame(data, index=None, columns=['dates', 'rh'])
plt.figure(figsize=(8,8))
g = sns.scatterplot(x='dates', y='rh', data=df, hue='dates', palette='colorblind', legend=False)
g.set_ylim(3.8, 2.1)
g.set_ylabel('Reflector Height (m)');

In [ ]:
plt.figure(figsize=(8,8))
df_group = df.groupby(['dates']).agg(['count'])
g = sns.scatterplot(data=df_group)
g.set_title('Number of values used in the daily average', size=16);

In [ ]:
filepath = f'{refl_code_loc}/Files/{station}-dailyavg.txt'
data = gnssrefl_helpers.read_allrh_file(filepath)
df = pd.DataFrame(data, index=None, columns=['dates', 'rh'])

In [ ]:
plt.figure(figsize=(8,8))
g = sns.scatterplot(x='dates', y='rh', data=df, legend=False)
g.set_ylim(3.6,2.3)
g.set_ylabel('Reflector Height (m)');

Three plots are returned. The first is all tracks. The second shows the number of tracks used in the daily average.Finally, the average RH each day for the year 2012.

This data shown in the last plot show you long-term accumulation as well as relatively small snow accumulation events. The overall 
plot is dominated by the large melt event in the summer.

**Things to think about:**

* Why do the number of useable tracks drop drastically at various times in the year?

* Why are the number of tracks retrieved in the summer days consistently higher in number than 
in other times of the year? What is different about the surface in the summer of 2012?

* How would you find out whether this year was anomalously large melt?  

* Try comparing the GNSS-IR results with the [validation data](https://tc.copernicus.org/articles/14/1985/2020/tc-14-1985-2020.pdf)

The original [J. Glaciology paper](https://www.kristinelarson.net/wp-content/uploads/2015/10/LarsonWahrKuipers_2015.pdf ) discussing this site.